# CGAN => conditional gan 
### this network solved the problem of randomness for the DGAN network by giving the network sense of the outputs by adding one-hot-encoding label vector to both generator and discriminator and give us the ability to control what the network generate so I used this architecture 

![CGAN](images/CGAN.png) 

In [ ]:
import tensorflow as tf 
import os 
from tensorflow.keras.layers import Dense , Conv2D ,concatenate, Conv2DTranspose , Flatten , Reshape  , BatchNormalization , Activation, Flatten ,LeakyReLU
from tensorflow.keras.models import Sequential 
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.datasets import mnist 
from tensorflow.keras.utils import to_categorical
import numpy as np 
import matplotlib.pyplot as plt 
import math 
import os 
import argparse


# Generator 

In [ ]:
class Generator(tf.keras.Model) : 
    def __init__(self) : 
        super().__init__()  
        self.dense_1 = Dense(7*7*128 ) 
        self.reshape = Reshape((7,7,128))

        
        self.sequential = Sequential([
             BatchNormalization() ,
             Activation('relu') , 
             Conv2DTranspose(128 , 5 , strides = 2 , padding = 'same' , use_bias = False) , 
             BatchNormalization() ,
             Activation('relu') ,
             Conv2DTranspose(64 , 5 , strides = 2  , padding = 'same' ,use_bias = False) ,
             BatchNormalization() ,
             Activation('relu') ,
             Conv2DTranspose(32 , 5   , padding = 'same' , use_bias = False) ,
             BatchNormalization() ,
             Activation('relu') ,
             Conv2DTranspose(1 , 5  , padding = 'same' , use_bias = False) , 
             Activation('sigmoid')
        ])
        
    def call(self , inputs ) :
        # we get noise vector and one hot label vector as inputs 
        X_vector , one_hot_vector = inputs
#         print(one_hot_vector.shape)
        X = concatenate([X_vector , one_hot_vector] , axis = 1) 
        X = self.reshape(self.dense_1(X ))
        return self.sequential(X)    
    
    

# Discriminator 

In [ ]:
class Discriminator(tf.keras.Model) : 
    def __init__(self) : 
        super().__init__() 
        self.label_dense = Dense(7*7*16) 
        self.reshape = Reshape((28 ,28 ,1)) 
        self.sequential = Sequential([
            LeakyReLU(.2), 
            Conv2D(32 , 5 , padding ='same' , strides = 2) , 
            LeakyReLU(.2),
            Conv2D(64 , 5 , padding ='same' , strides = 2) ,
            LeakyReLU(.2),
            Conv2D(128 , 5 , padding ='same' , strides = 2) ,
            LeakyReLU(.2),
            Conv2D(256 , 5 , padding ='same'  ) ,
            Flatten() , 
            Dense(1) , 
            Activation('sigmoid') 
        ])
        
        
    def call(self, inputs ) : 
        image ,one_hot_vector = inputs 
        # we do some sort of embedding to the label vector and reshape it to the image shape 
        lebel_embed = self.label_dense(one_hot_vector ) 
        label_embed = self.reshape(lebel_embed) 
        X = concatenate([image , label_embed]) 
        return self.sequential(X)
        
        

In [ ]:
def train_step(images , labels , generator , discriminator  , loss_fn , gen_optimizer , dis_optimizer):
    noise = tf.random.normal([64, 100])
    fake_labels = np.eye(10)[np.random.choice(10,64)]

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator((noise , fake_labels), training=True)

        real_output = discriminator((images , labels ) , training=True)
        fake_output = discriminator((generated_images , fake_labels) , training=True)

        gen_loss = loss_fn(fake_output , tf.ones_like(fake_output))
        disc_real_loss = loss_fn(real_output, tf.ones_like(real_output)) 
        disc_fake_loss = loss_fn(fake_output , tf.zeros_like(fake_output))
        disc_loss = disc_real_loss + disc_fake_loss
        
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    gen_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    dis_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
def plot_images(generator,
                noise_input,
                noise_class,
                show=False,
                step=0,
                model_name="gan"):
    os.makedirs(model_name, exist_ok=True)
    filename = os.path.join(model_name, "%05d.png" % step)
    images = generator.predict([noise_input, noise_class])
    print(model_name , " labels for generated images: ", np.argmax(noise_class, axis=1))
    plt.figure(figsize=(2.2, 2.2))
    num_images = images.shape[0]
    image_size = images.shape[1]
    rows = int(math.sqrt(noise_input.shape[0]))
    for i in range(num_images):
        plt.subplot(rows, rows, i + 1)
        image = np.reshape(images[i], [image_size, image_size])
        plt.imshow(image, cmap='gray')
        plt.axis('off')
    plt.savefig(filename)
    if show:
        plt.show()
    else:
        plt.close('all')

In [ ]:
def build_and_train_models(epochs):
    # load MNIST dataset
    (x_train, y_train ), (_, _) = mnist.load_data()

    # reshape data for CNN as (28, 28, 1) and normalize
    x_train = np.reshape(x_train, [-1, 28, 28, 1])
    x_train = x_train.astype('float32') / 255
    y_train = tf.keras.utils.to_categorical(y_train , 10)

    ds = tf.data.Dataset.from_tensor_slices((x_train , y_train) ) 
    ds = ds.shuffle(6000).batch(64)

    noise_class = np.eye(10)[np.arange(0, 16) % 10]
    cross_entropy = tf.keras.losses.BinaryCrossentropy()
    generator_optimizer = tf.keras.optimizers.Adam(1e-4)
    discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

    # build discriminator model
    discriminator = Discriminator()
    # build generator model
    generator = Generator()
    
    for epoch in range(epochs) : 
        for batch_images , labels  in ds : 
            train_step(images = batch_images , labels = labels , generator = generator  , discriminator = discriminator
            , loss_fn =cross_entropy , gen_optimizer = generator_optimizer , dis_optimizer =discriminator_optimizer )

        plot_images(generator ,tf.random.normal([16, 100]), noise_class , step = epoch  )


In [ ]:
build_and_train_models(10)